In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os
import cv2
import torch.optim as optim

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D, InputLayer
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential


from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ls_counts = 0
labels = np.zeros(5000)
for i in range(1,41):
  if i<10:
    i='0'+str(i)
  for i, _, k in os.walk('/content/drive/Shareddrives/high_classification/train/'+str(i)):  # store labels
    imgs = sorted(k)
    for filename in imgs:
      if str(filename)[-2] == 'p':
        img = cv2.imread(str(i)+'/'+str(filename))
        with open(str(i)+'/'+str(filename).replace('.jpg','.hytag'), 'r') as f:
          while True:                
            line = f.readline()
            if line:
                line = line.strip().split(',')
            
                # ROI coordinate extraction
                hytag_list = line 
                x=int(hytag_list[4])
                y=int(hytag_list[5])
                w=int(hytag_list[6])
                h=int(hytag_list[7]) 
                # label extraction
                
                if hytag_list[0] == 'LBUID_636802211114365653_0529': # Edema
                    labels[ls_counts] = 1
                elif hytag_list[0] == 'LBUID_636802210898995628_5475': # Normal 
                    labels[ls_counts] = 0

                roi = img[y:y+h, x:x+w]
                
                roi_resize = cv2.resize(roi, (64, 64))
                roi_resize = roi_resize[: ,:, 0]
                roi_resize_ravel = roi_resize.ravel()
                roi_resize_ravel = roi_resize_ravel.reshape(-1, 64, 64, 1)

                ls_counts += 1

                if ls_counts < 2:
                    roi_all = roi_resize_ravel
                else:
                    roi_all = np.vstack((roi_all, roi_resize_ravel))
            else:
                break

In [ ]:
labels = labels[:len(roi_all)]

In [ ]:
ls_counts = 0
t_labels = np.zeros(5000)
for i in range(41,51):
  for i, _, k in os.walk('/content/drive/Shareddrives/high_classification/train/'+str(i)):  # store labels
    imgs = sorted(k)
    for filename in imgs:
      if str(filename)[-2] == 'p':
        img = cv2.imread(str(i)+'/'+str(filename))
        with open(str(i)+'/'+str(filename).replace('.jpg','.hytag'), 'r') as f:
          while True:                
            line = f.readline()
            if line:
                line = line.strip().split(',')
            
                # ROI coordinate extraction
                hytag_list = line 
                x=int(hytag_list[4])
                y=int(hytag_list[5])
                w=int(hytag_list[6])
                h=int(hytag_list[7]) 
                # label extraction
                
                if hytag_list[0] == 'LBUID_636802211114365653_0529': # Edema
                    t_labels[ls_counts] = 1
                elif hytag_list[0] == 'LBUID_636802210898995628_5475': # Normal 
                    t_labels[ls_counts] = 0

                roi = img[y:y+h, x:x+w]
                
                roi_resize = cv2.resize(roi, (64, 64))
                roi_resize = roi_resize[: ,:, 0]
                roi_resize_ravel = roi_resize.ravel()
                roi_resize_ravel = roi_resize_ravel.reshape(-1, 64, 64, 1)

                ls_counts += 1

                if ls_counts < 2:
                    roi_all_t = roi_resize_ravel
                else:
                    roi_all_t = np.vstack((roi_all_t, roi_resize_ravel))
            else:
                break

In [ ]:
t_labels = t_labels[:len(roi_all_t)]
print(len(labels))
print(len(t_labels))
print(roi_all.shape)
print(roi_all_t.shape)

In [ ]:
np.save('train.npy',roi_all)
np.save('labels.npy',labels)
np.save('test.npy',roi_all_t)
np.save('t_labels.npy',t_labels)

In [ ]:
"""
---------------- 여기부터 Train ---------------------
"""

In [ ]:
x_train = np.load('/content/drive/Shareddrives/high_classification/data/train_x.npy')
y_train = np.load('/content/drive/Shareddrives/high_classification/data/train_y.npy')
x_valid = np.load('/content/drive/Shareddrives/high_classification/data/validation_x.npy')
y_valid = np.load('/content/drive/Shareddrives/high_classification/data/validation_y.npy')

In [ ]:
from sklearn.utils import shuffle 

y_train = y_train.astype(np.uint8)
y_valid = y_valid.astype(np.uint8)

x_train, y_train = shuffle(x_train, y_train, random_state=13)
x_valid, y_valid = shuffle(x_valid, y_valid, random_state=13)


In [ ]:
x_train.shape

In [ ]:
x_train[0].shape

In [ ]:
a = np.zeros((64, 64, 3))
a[:, :, 0] = x_train[1].reshape((64, 64, 1))
# a[:, :, 1] = x_train[2]
# a[:, :, 2] = x_train[3]

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

In [ ]:
# class CNN(nn.Module):
#   def __init__(self):
#     super(CNN,self).__init__()
#     self.layer1 = nn.Sequential(
#         nn.Conv2d(1,3,5),
#         nn.ReLU(),
#         nn.MaxPool2d(4))
#     self.layer2 = nn.Sequential(
#         nn.Conv2d(3,6,4),
#         nn.ReLU(),
#         nn.MaxPool2d(6))
#     self.layer3 = nn.Sequential(
#         nn.Linear(6*10*10,300),
#         nn.ReLU(),
#         nn.Linear(300,2))

#   def forward(self, x):
#     out = self.layer1(x)
#     out = self.layer2(out)
#     out = out.view(6*10*10, -1)
#     out = self.layer3(out)
#     return out

# keras model
def CNN_base_model(learning_rate=1e-3):

    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())  # 3d array from CNN layer to 1d array 
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.SGD(),
                  metrics=['accuracy'])
    return model

In [ ]:
# net = CNN()
# print(net)

model = CNN_base_model()
model.summary()

In [ ]:
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.01)

In [ ]:
# loader = DataLoader(train)
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
batch_size = 16

train_datagen = ImageDataGenerator(rescale=1./255)  # Normalization
valid_datagen = ImageDataGenerator(rescale=1./255)  # Normalization

train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size, seed=13)
valid_generator = valid_datagen.flow(x_valid, y_valid, batch_size=batch_size, seed=13)

In [ ]:
# for epoch in range(20):   

#     running_loss = 0.0
#     for data in loader:
#         optimizer.zero_grad()
#         outputs = net(data.reshape(1,1,64, 64))
#         loss = criterion(outputs, train_labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         print('[Epoch:{}] cost = {}'.format(epoch+1, running_loss))


epochs = 100

# mc = ModelCheckpoint('mc/0709-all-ori-SGD.h5', monitor='val_accuracy', mode='max', save_best_only=True)

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=int(x_valid.shape[0] / batch_size),
    steps_per_epoch= int(x_train.shape[0] / batch_size),)
    # callbacks=[mc]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')